# Visual Saliency Analysis 
This Jupyter notebook contains several data cells that feature approaches to analyze visual saliency in an eye tracking experiment. The eye tracking experiment involved of 1,250 stimuli (Microsoft Coco Images: 1000 different images and 250 (1/4) of them repeated). These experimental conditions are the same as in the well-known experiment within the NSD dataset by Allen et al. (2022). Allen et al. (2022) systematically recorded only the voxel data of participants performing the same experiment while lying in an fMRI scanner. In contrast, our study captured the critical, missing ground truth eye-tracking data. We recorded the eye movements of 23 participants as they viewed the same images used by Allen et al. (2022). The focus of this analysis is on the participants' eye movements, particularly fixations, during image perception.

Participants performed a continuous recognition task, which required them to remember and identify as many stimuli as possible. This indirect task design allowed for natural image exploration, yielding in more authentic eye-tracking data that better represents real-world visual perception.



Written by Lisa Heinemann  
Last edited: 20/12/2025


## A. Creation of Fixation Maps and Saliency Maps 

This notebook creates fixation and saliency maps of participants looking at a certain image by accessing the concatenated DataFrame that includes all paticipant data (nr 1 - 23).

### *I. Import necessary libaries for this code cell* 


In [1]:
import h5py
# Pathlib libary is neccessary because we are working with the objects 
from pathlib import Path

# Needed for the image processing within the scatterplot 
from PIL import Image

# To generate saliency maops
import statsmodels.api as sm
# Pandas libary is needed as well, since we are working with dataframes
import pandas as pd
# OpenCV is needed to read the images
import cv2
import os
import numpy as np



# Matplotlib: Needed to plot the images 
import matplotlib.pyplot as plt


### *II. Base directory, data directory, save directory and image directory setup* 

In [2]:
# Create a Path object for the base directory
#base_dir = Path('~/saliency-nsd-priv').expanduser()

############################################################################################
# # Create a Path object for the base directory pub
# TODO: Change to the correct Path.
base_dir = Path('/gpfs01/bartels/group/lheinemann/saliency-nsd-pub').expanduser()

# Check if the base directory exists with an assert 
assert base_dir.exists(), f'{base_dir} does not exist'
# Print the base directory
print(base_dir)

# Combine paths to create new Path objects for the data directory and the save directory

# Adapted to this Jupyter notebook 

# Path to the concatenated DataFrame 'df_expanded_data_subjs_concatenated_all.csv' containing all experimental data
# One line represents one fixation of one subject in the DataFrame 
data_dir = base_dir / 'data/preprocessed/df_expanded_data_subjs_concatenated_all.csv'
# To load the actual jpg images responding to their name stored in img_id in the dataframe 
img_dir = base_dir /'stimuli/shared1000'
# The fixation maps are stored in the save_dir
save_dir = base_dir / 'data/fixation_and_saliency_maps'

# TODO: Delete if not needed anymore

# Add hdf5 file path to store the values of the saliency maps
hdf5_file_path_all_and_first_fixations = save_dir / 'saliency_map_all_and_first_fixations.h5'

# Check if the data and save directories exist with assert statements, if not assertion error is raised
assert data_dir.exists(), f'{data_dir} does not exist'
assert save_dir.exists(), f'{save_dir} does not exist'
assert img_dir.exists(), f'{img_dir} does not exist'

# Debug check: Print statement of path 
print(data_dir)
print(save_dir)
print(img_dir)
print(hdf5_file_path_all_and_first_fixations)

# Print the working directory to double check: should be /gpfs01/bartels/user/lheinemann/saliency-nsd/code/code_analysis_saliency-nsd/1_data-preprocessing/creation_of_dataframes
print (os.getcwd())


/gpfs01/bartels/group/lheinemann/saliency-nsd-pub
/gpfs01/bartels/group/lheinemann/saliency-nsd-pub/data/preprocessed/df_expanded_data_subjs_concatenated_all.csv
/gpfs01/bartels/group/lheinemann/saliency-nsd-pub/data/fixation_and_saliency_maps
/gpfs01/bartels/group/lheinemann/saliency-nsd-pub/stimuli/shared1000
/gpfs01/bartels/group/lheinemann/saliency-nsd-pub/data/fixation_and_saliency_maps/saliency_map_all_and_first_fixations.h5
/gpfs01/bartels/user/lheinemann/saliency-nsd-priv/code/code_analysis_saliency-nsd/2_analysis/VisualizationOfFixationsAndSaliencyMaps


## 1. Complete run through all 1000 images. 

This code cell: 
- Generates averaged saliency maps over all participants 

- 1. Loads the tsv file that contains a list of the distinct image ids and loads the file into a DataFrame
- 2. Defines the function 'get_block_and_trial_id(img_id)' that returns the current Block Number, Trial Number, Subject Number and the coherent filtered DataFrame according to the current element of the iteration
- 3. Defines the function 'transform_coordinates' to apply the linear tranformation that maps the coordinates from the experimental screen to their corresponding positions within the image file. The transformed coordinates are stored in the dataframe df_transformed.
- 4. Processes the data
    *Linear Transformation of Coordinates*
    - Converts experimental screen coordinates into corresponding image file coordinates.

    *Visualization of All Fixations* 
    - Plots all fixations from all participants as circles on the image

    *Fixation Filtering*
    - Removes fixations >50ms (according to the recommendations of Rothkegel et al. (2019))

    *First Fixation Extraction*
    - Extracts the first valid fixation (>50ms) for each subject

    *Linear Transformation of First Fixation Coordinates*
    - Maps first fixation coordinates from the experimental screen to image file coordinates 
    - Stores the transformed data in *df_firstFixations_transformed* 
    
    *First fixation visualzation*
    - Plots the first fixation (>50ms, *N*= 23) for all participants as circles on the image and saves the file 
  
   
- 5. Calculates the data and visualizes and saves the saliency maps in .npz and .png files

    *Saliency Map Calculation and Output*
    - Bandwidth Estimation: 
        * Determines the bandwith of the convolution kernel for KDE (Kernel Density Estimation) using maximum likelihood cross-validation (cv_ml).
    -  df_transformed or df_first_fixations_transformed contains the adjusted fixation points (c1, c2)
    - Visualization: 
        * Generates averaged saliency maps for: 
            * All fixations of all participants 
            * First fixations (>50ms) of all participants
        * Saves the saliency maps as .csv files and the visualizations as .png files
    

Further Details of the Saliency Estimation 

*Bandwidth Estimation:*

Purpose:
- The bandwidth controls the smoothness of the KDE, estimating saliency across image regions (pixels).
Cross-Validation Techniques:
- cv_ml (Maximum Likelihood Cross-Validation):
    - Default and recommended method.
    - More accurate, especially for small sample sizes.
- cv_ls (Least Squares Cross-Validation):
    - Less computationally intensive.
    - Not recommended for small datasets due to lower accuracy.

*Method Notes:*

The cv_ml method is used for all bandwidth estimations in this process as it is generally preferred for its precision.



In [ ]:
## IV. Load all the img_ids from the tsv file of the nsd dataset, stored at: 
# /gpfs01/bartels/user/lheinemann/saliency-nsd-priv/stimuli

# Load the name of the used img_ids of the nsd dataset name: shared1000.tsv.txt

# Define the path to the tsv file
tsv_file_path = '/gpfs01/bartels/user/lheinemann/saliency-nsd-priv/stimuli/shared1000.tsv.txt'

# Load the tsv file into a DataFrame
img_nums_df = pd.read_csv(tsv_file_path, sep='\t', header=None)  # Use header=None because in the tsv file there's no header row
# Otherwise the first row would be interpreted as the header row and the first image_id (25951) would be missing

# Rename the first column to 'img_id'
img_nums_df.rename(columns={img_nums_df.columns[0]: 'img_id'}, inplace=True)

# Set the maximum number of rows to display, so that all rows are displayed
pd.set_option('display.max_rows', 1000)

# Print the first rows of the DataFrame
print(img_nums_df.head())

#################### Function definition to get the block number and trial number for a given image_id ####################

# Function to get the block number and trial number for a given image_id
def get_block_and_trial(img_id):

    # Filter the DataFrame based on image_id
    df_filtered = df_expanded_all[df_expanded_all['Image Id'].str.strip("'") == img_id]

    # For all rows in df_filtered retrieve the sub_num, currentBlockNumber, trial_num, and the image_id
    for index, row in df_filtered.iterrows():

        # Retrieve the block number and trial number
        currentBlockNumber = row['Block Number']
        trial_num = row['Trial Number']
        sub_num = row['Subject Number']
        
        # Debug option: Print the subject number, block number, trial number, and image_id
        # print(row['Subject Number'], row['Block Number'], row['Trial Number'], row['Image Id'])
    
    return currentBlockNumber, trial_num, sub_num, df_filtered
##############################################################################################################################################################################

########## Function definition to transform the fixation coordinates from the original position relative to the experimental screen to match the image dimensions ############

# Linear transformation of the fixation x- and y-coordinates (indicating the position on the experimental screen (1920 x 1080)), to the position on the image (e.g. 258 x 258)
def transform_coordinates(x_coords, y_coords):
    
    # Get the original and cropped image dimensions
    original_img_size = 425
    cropped_img_size = 258

    # Calculate the offset for centering the image on a 1920 x 1080 screen
    offset_x = (1920 - cropped_img_size) / 2
    offset_y = (1080 - cropped_img_size) / 2

    # Adjust the fixation coordinates by subtracting the offset
    x_coords = x_coords - offset_x
    y_coords = y_coords - offset_y

    # Scale the adjusted coordinates to match the cropped image dimensions
    x_coords_transformed = (original_img_size / cropped_img_size) * x_coords
    y_coords_transformed = (original_img_size / cropped_img_size) * y_coords

    return x_coords_transformed, y_coords_transformed

##############################################################################################################################################################################
################# Main Loop to create fixation maps and saliency maps for all fixations of each subject on the first presentation of the image ###############################
##############################################################################################################################################################################

# Loop through each img_id in the DataFrame
# img_num are the unique image numbers that can be entered to complete the img_id in the DataFrame. Alternatively the distinct images can be viewed in the 
# img_dir: /gpfs01/bartels/user/lheinemann/saliency-nsd-priv/stimuli/shared1000



# Create the h5py file to store the saliency maps for all fixations
with h5py.File(hdf5_file_path_all_and_first_fixations, 'w') as hdf5_file:

#TODO: Create the same for first fixations, or TODO: check whether I really need this because in the h5group the file was already created
#with h5py.File(hdf5_file_path_first_fixations, 'w') as hdf5_file:

    # TODO: If code runs change back to the original range the images were processed in the first run of the code
    #for index, row in img_nums_df.head(3).iterrows():
    #TODO: Change the range to the desired amount of images to be processed
    for index, row in img_nums_df.iterrows():
        img_num = row['img_id']

        # Get the image path for the given image_id
        img_id = f'nsd_stimulus_id_{img_num}.jpg'

        # Debug check: Print processing statement of the img_id
        # print(f'Processing image {img_id}')

        # Load the concatenated DataFrame df_expanded_data_subjs_concatenated_all.csv
        df_expanded_all = pd.read_csv(data_dir)

        # Debug check: Print the first rows of the loaded DataFrame
        #print(df_expanded_all.head())

        ############################################################################################################
    
        # Call the function to get the block number and trial number for the given image_id and enter the results
        # to the dataframe
        currentBlockNumber, trial_num, sub_num, df_filtered = get_block_and_trial(img_id)
        
        ############################################################################################################

        # Construct the image path
        img_path = img_dir / f'{img_id}'
        assert img_path.exists(), f'{img_path} does not exist'

        # Filter the DataFrame for the specific image_id
        df_filtered = df_expanded_all[df_expanded_all['Image Id'].str.strip("'") == img_id]

        # Load the image using cv2 - OpenCV
        img = cv2.imread(str(img_path))

        # Check if the image was loaded correctly
        if img is None:
            raise ValueError("Image not found or unable to load.")

        # Convert the image from BGR to RGB (OpenCV loads images in BGR format)^
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Extract x and y coordinates from the filtered DataFrame
        x_coords = df_filtered['Fixation X Coordinate'].values
        y_coords = df_filtered['Fixation Y Coordinate'].values

        ########################### Linear Transformation of Fixation Coordinates - All Fixations ###########################
        x_coords_transformed, y_coords_transformed = transform_coordinates(x_coords, y_coords)

        # Create a new DataFrame with the transformed coordinates so that the saliency map can be plotted with 
        # the correct fixation positions
        df_transformed = df_filtered.copy()
        df_transformed['Fixation X Coordinate Transformed'] = x_coords_transformed
        df_transformed['Fixation Y Coordinate Transformed'] = y_coords_transformed

        #####################################################################################################################
        # Draw all fixation data points on the image
        #####################################################################################################################

        ######################## Using OpenCV to load the image and draw fixation points ####################################
        viz_img = img.copy()
        txt_offset = 7

        # Draw a circle at each fixation point on the overlay and enumerate them
        for i, (x, y) in enumerate(zip(x_coords_transformed, y_coords_transformed)):

            # Debug option - Print the coordinates of the fixation points
            # print(f'Drawing text for point {i+1} at coordinates ({int(x)}, {int(y)})')

            # Draw the outer black circle
            cv2.circle(viz_img, (int(x), int(y)), radius=5, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)
            
            # Option: Add enumeration text to the fixation points
            #cv2.putText(viz_img, str(i+1), (int(x) + txt_offset, int(y)),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

        # Display the image with fixation points using Matplotlib
        plt.imshow(viz_img)

        # Show the name of the image in the title and the subject number, block number and trial number
        plt.title(f'Image: {img_path.name}')
        plt.axis('off')  # Hide the axis
        # Debug option to show the plot in the console
        # plt.show()

        # Save the visualization image with fixation points to a png file
        save_path = save_dir / f'all_fixations_nsd_stimulus_id_unfiltered_{img_num}' 
        cv2.imwrite(f'{save_path}.png', cv2.cvtColor(viz_img, cv2.COLOR_RGB2BGR))

        # Clear the current figure to avoid overlap in the next iteration
        plt.clf()
        # Close the figure to ensure it is not displayed
        plt.close()

        ####################################################################################################################
        ######## Eliminate the fixations < 50ms from the df_filtered DataFrame (as suggested by Rothkegel et al. (2019))####

        # Eliminate fixations with durations < 50ms = < 0.05s (since the fixation duration is stored in seconds in the DataFrame)
        df_filtered = df_filtered[df_filtered['Fixation Duration'] >= 0.05]

        # Select the first fixation for each subject on the first presentation of the image
        df_first_fixations = df_filtered.groupby(['Subject Number', 'Image Id']).first().reset_index()

        # Debug option: List the amount of rows of the filtered DataFrame - indicating the amount of fixations in the trial
        #print(len(df_first_fixations))
        #print(df_first_fixations)

        # TODO: Delete this code part because this data processing is not neccessary and to time and data space consuming
        # Save the first fixations DataFrame to a CSV file
        # df_first_fixations.to_csv(save_dir / f'first_fixations_nsd_stimulus_id_{img_num}.csv', index=False)

        # Only load x and y coordinates from the filtered DataFrame
        x_coords = df_first_fixations['Fixation X Coordinate'].values
        y_coords = df_first_fixations['Fixation Y Coordinate'].values

        # Debug option: Print the x and y coordinates of the fixations with a duration of at least 50ms
        # print(x_coords)
        # print(y_coords)

        ####################################################################################################################

        # Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080)
        # to the position on the image (e.g. 258 x 258) see function definition of the for-loop above transform_coordinates

        ########################### Linear Transformation of Fixation Coordinates - All Fixations ###########################
        x_coords_transformed, y_coords_transformed = transform_coordinates(x_coords, y_coords)

        # Create a new DataFrame with the transformed coordinates so that the saliency map can be plotted with
        # the correct fixation positions
        df_first_fixations_transformed = df_first_fixations.copy()
        df_first_fixations_transformed['Fixation X Coordinate Transformed'] = x_coords_transformed
        df_first_fixations_transformed['Fixation Y Coordinate Transformed'] = y_coords_transformed

        # Using OpenCV to draw circles on the image, and imitating transparency
        viz_img = img.copy()
        txt_offset = 7

        # Draw a circle at each fixation point on the overlay and enumerate them
        for i, (x, y) in enumerate(zip(x_coords_transformed, y_coords_transformed)):
            # Debug option: Print the coordinates of the fixation points 
            # print(f'Drawing text for point {i+1} at coordinates ({int(x)}, {int(y)})')
            # Draw the outer black circle
            cv2.circle(viz_img, (int(x), int(y)), radius=5, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)
            
            # Option: Add enumeration text
            #cv2.putText(viz_img, str(i+1), (int(x) + txt_offset, int(y)),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

        plt.imshow(viz_img)

        # Show the name of the image in the title and the subject number, block number and trial number
        plt.title(f'First Fixations: {img_path.name}')
        plt.axis('off')  # Hide the axis

        # Save the visualization image with fixation points to a png file
        save_path = save_dir / f'first_fixations_nsd_stimulus_id{img_num}' 
        cv2.imwrite(f'{save_path}.png', cv2.cvtColor(viz_img, cv2.COLOR_RGB2BGR))

        # Clear the current figure to avoid overlap in the next iteration
        plt.clf()
        # Close the figure to ensure it is not displayed
        plt.close()
        
        """ 
        # Debug option: Display the image with fixation points using Matplotlib
        plt.show()
        """

        # Debug check: If saved send a confirmation message
        #print(f'Saved visualization to {save_path}.png')

    ####################################################################################################################################
    #             Create saliency maps for all fixations of each subject on the first presentation of the image
    ####################################################################################################################################

        # Use all fixations of the participants while perceive the image (first time?). Because this will provide a more comprehensive representation of the AOI's across all participants 
        # and an accurate saliency map.

        ### Load the relevant data to generate a saliency map for the image (e.g. nsd_stimulus_id_3050.jpg) ###

        # Load the image using cv2 - OpenCV
        img = cv2.imread(str(img_path))

        # Check if the image was loaded correctly
        if img is None:
            raise ValueError("Image not found or unable to load.")

        # Convert the image from BGR to RGB (OpenCV loads images in BGR format)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080), 
        # was already done in the previous cells and is stored in df_transformed

        # df_transformed contains the adjusted fixation points (c1, c2)
        c1 = df_transformed['Fixation X Coordinate Transformed'].values
        c2 = df_transformed['Fixation Y Coordinate Transformed'].values

        # Estimate the bandwidth using KDEMultivariate with 'cv_ml' method
        dens_u = sm.nonparametric.KDEMultivariate(data=[c1, c2], var_type='cc', bw='cv_ml')

        # Create a meshgrid of the image dimensions
        x_sz, y_sz = 425, 425
        Ys, Xs = np.mgrid[0:x_sz, 0:y_sz]

        # Prepare the data for prediction
        data_predict = np.vstack([Xs.ravel(), Ys.ravel()]).T

        # Calculate the density values for each pixel
        density = dens_u.pdf(data_predict)

        # Debug option: Print the sum of the density values to check whether the probability density function is close/ equal to 1
        #print(density.sum())

        # Debug option: Print the density values for each pixel
        #print(f"Density-values: {density}")

        # Reshape the density values to match the image dimensions
        saliency_map = density.reshape(x_sz, y_sz)

        # Option to normalize the saliency map to have values between 0 and 1 - but not desired for now to be able to see intergroup differences
        # saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())

        # Debug option: Print the values of the saliency map
        #print(f"Saliency Map: {saliency_map}")

        ###################################################################################################################################################
        ####################### Save the saliency map to a np.savez array #######################

        # (containing the saliency values at the distinct pixel areas and reduce it to float32 single precision file
        # Reducing the decimal places ###################

        # Convert the saliency map to a single precision float array to reduce the file size
        # The large saliency map array should have the following shape: (1000, 425, 425)
        # 1000 = indicating the amount of distinct images in shared1000. 425 x 425 = the pixel dimensions of the image
        large_saliency_map = saliency_map.astype('single')
        # Assign the image id to the saliency map ids to be able to match the saliency map to the image
        #map_ids = [img_id] * len(large_saliency_map)

        group_all_fixations = hdf5_file.require_group('all_fixations')
        group_all_fixations.create_dataset(f'image_{img_num}', data=large_saliency_map)

        ###########################################################################
        # # Plot the saliency map

        plt.imshow(saliency_map, cmap='hot', interpolation='nearest')
        plt.colorbar()
        plt.title('Saliency Map - All fixations: ' + img_id, fontsize= 10)
        # Save the plot to a png file
        save_path = save_dir / f'saliency_map_all_fixations_nsd_stimulus_id_{img_num}'
        plt.savefig(f'{save_path}.png')
        # Clear the current figure to avoid overlap in the next iteration
        plt.clf()
        # Close the figure to ensure it is not displayed
        plt.close()

        """ 
        # Debug option: Display the saliency map of all fixations using Matplotlib
        plt.show()
        """
        ################### Overlay the saliency map on the image ###################

        # Plot the image
        plt.imshow(img, cmap='gray', interpolation='nearest')

        # Overlay the saliency map with a trasparency of alpha = 0.7, TODO: Change to 0.5 if the code works s
        plt.imshow(saliency_map, cmap='hot', alpha=0.6, interpolation='nearest')

        # Add a colorbar and title
        plt.colorbar()
        plt.title('Saliency Map - All fixations: ' + img_id, fontsize=10)

        # Save the plot to a png file
        save_path = save_dir / f'saliency_map_overlay_all_fixations_nsd_stimulus_id_{img_num}'
        plt.savefig(f'{save_path}.png')

        # Debug check: If saved send a confirmation message
        # print(f'Saved overlay to {save_path}.png')
        # Clear the current figure to avoid overlap in the next iteration
        plt.clf()
        # Close the figure to ensure it is not displayed
        plt.close

        """ 
        # Debug option: Display the image with the saliency map of all fixations using Matplotlib
        plt.show()
        """

    ####################################################################################################################################
    # Create a saliency map for the first fixation of each subject on the first presentation of the image
    ####################################################################################################################################

        ### Load the relevant data to generate a saliency map for an image of shared1000 (e.g. nsd_stimulus_id_3050.jpg) ###
        # Used img ids are stored in the shared1000.tsv.txt file

        # Load the image using cv2 - OpenCV
        img = cv2.imread(str(img_path))

        # Check if the image was loaded correctly
        if img is None:
            raise ValueError("Image not found or unable to load.")

        # Convert the image from BGR to RGB (OpenCV loads images in BGR format)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080), 
        # was already done in the prior parts of this code cell and is stored in df_transformed

        # df_first_fixations_transformed contains the adjusted fixation points (c1, c2)
        c1 = df_first_fixations_transformed['Fixation X Coordinate Transformed'].values
        c2 = df_first_fixations_transformed['Fixation Y Coordinate Transformed'].values

        # Estimate the bandwidth using KDEMultivariate with 'cv_ml' method
        dens_u = sm.nonparametric.KDEMultivariate(data=[c1, c2], var_type='cc', bw='cv_ml')

        # Create a meshgrid of the image dimensions
        x_sz, y_sz = 425, 425
        Ys, Xs = np.mgrid[0:x_sz, 0:y_sz]

        # Prepare the data for prediction using the ravel function 
        data_predict = np.vstack([Xs.ravel(), Ys.ravel()]).T

        # Calculate the log density values for each pixel
        density = dens_u.pdf(data_predict)

        # Debug option: Print the sum of the density values to check whether the probability density function is close/ equal to 1
        # print(density.sum())

        # Debug chekc: Print the density values for each pixel
        # print(f"Density-values: {density}")

        # Reshape the density values to match the image dimensions
        saliency_map_first_fixations = density.reshape(x_sz, y_sz)

        # Normalize the saliency map to have values between 0 and 1 / Normalization is not desired for the saliency map, to be able to see intergroup differences
        # saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())

        # Debug check: Print the values of the saliency map
        # print(f"Saliency Map: {saliency_map}")


        ##################################################################################################################################################
        ####################### Save the saliency map to a np.savez array #######################
        
        # (Containing the saliency values at the distinct pixel areas and reduce it to float32 single precision file
        # Reducing the decimal places ###################

        # Convert the saliency map to a single precision float array to reduce the file size
        # The large saliency map array should have the following shape: (1000, 425, 425)
        # 1000 = indicating the amount of distinct images in shared1000. 425 x 425 = the pixel dimensions of the image
        large_saliency_map_first_fixations = saliency_map_first_fixations.astype('single')
       
        # Assign the image id to the saliency map ids to be able to match the saliency map to the image
        #map_ids = [img_id] * len(large_saliency_map_first_fixations)
        group_first_fixations = hdf5_file.require_group('first_fixations')
        group_first_fixations.create_dataset(f'image_{img_num}', data=large_saliency_map_first_fixations)

        ###########################################################################

        # Display the saliency map
        plt.imshow(saliency_map, cmap='hot', interpolation='nearest')
        plt.colorbar()
        plt.title('First Fixations - Saliency Map of: ' + img_id, fontsize= 8)

        # Save the plot to a png file
        save_path = save_dir / f'saliency_map_first_fixations_nsd_stimulus_id_{img_num}'
        plt.savefig(f'{save_path}.png')
        # Clear the current figure to avoid overlap in the next iteration
        plt.clf()
        # Close the figure to ensure it is not displayed
        plt.close()

        """ 
        # Debug option: Display the saliency map of the first fixations using Matplotlib
        plt.show()
        """

        ################### Overlay the saliency map on the image ##################

        # Plot the image
        plt.imshow(img, cmap='gray', interpolation='nearest')

        # Overlay the saliency map with a trasparency of alpha = 0.7, TODO: Change to 0.5 if the code works s
        plt.imshow(saliency_map, cmap='hot', alpha=0.6, interpolation='nearest')

        # Add a colorbar and title
        plt.colorbar()
        plt.title('First Fixations - Saliency Map of: ' + img_id, fontsize=8)

        # Save the plot to a png file
        save_path = save_dir / f'saliency_map_overlay_first_fixations_nsd_stimulus_id_{img_num}'
        plt.savefig(f'{save_path}.png')
        # Clear the current figure to avoid overlap in the next iteration
        plt.clf() 
        # Close the figure to ensure it is not displayed
        plt.close()
        
        """ 
        # Debug option: Display the image with the saliency map of the first fixations using Matplotlib
        plt.show()
        """
# Print a final statement to indicate that the script has finished
print('Script finished successfully and has created the fixation maps and saliency maps and the corresponding HDF file (storing the saliency values for distinct images) for all 1000 images.')



   img_id
0    2951
1    2991
2    3050
3    3078
4    3147


/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expanded_all = pd.read_csv(data_dir)
/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expanded_all = pd.read_csv(data_dir)
/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expanded_all = pd.read_csv(data_dir)
/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expanded_all = pd.read_csv(data_dir)
/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_expanded_all = pd.read_csv(data_dir)
/tmp/ipykernel_2333589/3805636617.py:97: DtypeWarning: Columns (13) have mixed t

Script finished successfully and has created the fixation maps and saliency maps and the corresponding HDF file (storing the saliency values for distinct images) for all 1000 images.


## Check whether h5py file can be opened: 

In [6]:
# Check whether the HDF5 file is filled with the correct data
# Open the HDF5 file in read mode
with h5py.File(hdf5_file_path_all_and_first_fixations, 'r') as hdf5_file:
    # List all groups
    print("Groups in the HDF5 file:")
    for group_name in hdf5_file:
        print(group_name)
        
        # List all datasets in the group
        group = hdf5_file[group_name]
        for dataset_name in group:
            print(f"  Dataset: {dataset_name}")
            
            # Print the shape and dtype of the dataset
            dataset = group[dataset_name]
            print(f"    Shape: {dataset.shape}")
            print(f"    Dtype: {dataset.dtype}")
            
            # Optionally, print the data (or a subset of it)
            # print(f"    Data: {dataset[:]}")

# Example of accessing a specific dataset for all fixations (e.g. image_2951)
with h5py.File(hdf5_file_path_all_and_first_fixations, 'r') as hdf5_file:
    dataset = hdf5_file['all_fixations/image_2951']
    print("Data for all_fixations/image_2951:")
    print(dataset[:])
# Example of accessing a specific dataset for first fixations (e.g. image_2951)
with h5py.File(hdf5_file_path_all_and_first_fixations, 'r') as hdf5_file:
    dataset = hdf5_file['first_fixations/image_2951']
    print("Data for first_fixations/image_2951:")
    print(dataset[:])
# 2nd example of accessing a specific dataset for all fixations 
with h5py.File(hdf5_file_path_all_and_first_fixations, 'r') as hdf5_file:
    dataset = hdf5_file['all_fixations/image_20739']
    print("Data for all_fixations/image_20739:")
    print(dataset[:])
# 2nd example of accessing a specific dataset for first fixations 
with h5py.File(hdf5_file_path_all_and_first_fixations, 'r') as hdf5_file:
    dataset = hdf5_file['first_fixations/image_20739']
    print("Data for first_fixations/image_20739:")
    print(dataset[:])

Groups in the HDF5 file:
all_fixations
  Dataset: image_10007
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10047
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10065
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10106
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10108
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10245
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10394
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10472
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10508
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10587
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10601
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10611
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_10908
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_11160
    Shape: (425, 425)
    Dtype: float32
  Dataset: image_11334
    Shape: (425, 425)
    Dtype: float32
 

## 2. Further cells of this notebook are useful, if you want to (re)create the visualization of a certain image: 

- Fixation Map of all Fixations (Option: Enumerate fixations)
- Fixation Map of only the first fixations during the first presentation of the image for the participants (Option: Enumerate fixations)
- Saliency Map of all Fixations 
- Saliency Map of only the first fixations during the first presentation of the image for the participants
- Print the fixations on a scatter plot, representing the dimensions of the experimental screen 

### 2.1. Visualization of fixations on an image. -> Print fixation on the  image
The stimulus-on times are obtained from the Matlab file using Pandas Dataframe (stimulus on and stimulus off time). A trial consists of 3s stimulus on and 1s stimulus off.  

*Code description:* This code prints the fixation within the Stimulus-On time on the distinct image presented to the participant during that time. 



#### 2.1.1 Setup of Visualization: This code cell needs to be runned at first. Entering an image id of shared1000. 

In [ ]:
# TODO: Change the subject_nr, block_nr, and trial_nr to get the image path for a different trial

# Example usage: Get the image path for trial 1 for subj 1
# In the DataFrame, each particpant had 1250 trials but the inedixing of the trial_num is related to the currentblockNumber
# Meaning trial_num goes from1-250 for every block, like in the Matlab file  

# Access all img_ids consistent in the tsv file and run all the code cells above in a for loop to get the fixation maps for all images

# Define the image number
img_num = 3952
img_id = f'nsd_stimulus_id_{img_num}.jpg'
# print statement of the img_id
print(img_id)
# Example usage: Get the image path for a given image_id
# img_id = 'nsd_stimulus_id_4787.jpg'

# Load the concatenated DataFrame df_expanded_data_subjs_concatenated_all.csv
df_expanded_all = pd.read_csv(data_dir)

# Print the first rows of the loaded DataFrame
print(df_expanded_all.head())

# Function to get the block number and trial number for a given image_id
def get_block_and_trial(img_id):
    # Filter the DataFrame based on image_id
    df_filtered = df_expanded_all[df_expanded_all['Image Id'].str.strip("'") == img_id]
    
    # Debug Option: Remove duplicate rows if any
    df_filtered = df_filtered.drop_duplicates()

    # List the amount of rows of the filtered DataFrame - indicating the amount of fixations in the trial
    print("Length of Dataframe:", len(df_filtered))
    print(df_filtered)

    # for all rows in df_filtered print the sub_num, currentBlockNumber, trial_num, and the image_id
    for index, row in df_filtered.iterrows():

        # Retrieve the block number and trial number
        currentBlockNumber = row['Block Number']
        trial_num = row['Trial Number']
        sub_num = row['Subject Number']
        
        print(row['Subject Number'], row['Block Number'], row['Trial Number'], row['Image Id'])
    
    return currentBlockNumber, trial_num, sub_num, df_filtered

In [4]:
# Construct the image path
img_path = img_dir / f'{img_id}'
assert img_path.exists(), f'{img_path} does not exist'

# Filter the DataFrame for the specific image_id
df_filtered = df_expanded_all[df_expanded_all['Image Id'].str.strip("'") == img_id]

# Load the image using cv2 - OpenCV
img = cv2.imread(str(img_path))

# Check if the image was loaded correctly
if img is None:
    raise ValueError("Image not found or unable to load.")

# Convert the image from BGR to RGB (OpenCV loads images in BGR format)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Extract x and y coordinates from the filtered DataFrame
x_coords = df_filtered['Fixation X Coordinate'].values
y_coords = df_filtered['Fixation Y Coordinate'].values

########################### Linear Transformation of Fixation Coordinates ###########################

# Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080)
# to the position on the image (e.g. 258 x 258)

# Get the original and cropped image dimensions
original_img_size = 425
cropped_img_size = 258

# Calculate the offset for centering the image on a 1920 x 1080 screen
offset_x = (1920 - cropped_img_size) / 2
offset_y = (1080 - cropped_img_size) / 2

# Adjust the fixation coordinates by subtracting the offset
x_coords = x_coords - offset_x
y_coords = y_coords - offset_y

# Scale the adjusted coordinates to match the cropped image dimensions
x_coords_transformed = (original_img_size / cropped_img_size) * x_coords
y_coords_transformed = (original_img_size / cropped_img_size) * y_coords

# Create a new DataFrame with the transformed coordinates so that the saliency map can be plotted with 
# the correct fixation positions
# LOOK AT PART B OF THE NOTEBOOK FOR THE PLOTTING OF THE SALIENCY MAP
df_transformed = df_filtered.copy()
df_transformed['Fixation X Coordinate Transformed'] = x_coords_transformed
df_transformed['Fixation Y Coordinate Transformed'] = y_coords_transformed

##### 2.1.2.1 Visualization of Fixations on Images (Using OpenCV) - Case I: Visualization of all fixations in a DataFrame 

In [ ]:
######################## Using OpenCV to draw circles on the image, and imitating transparency  ########################
viz_img = img.copy()
txt_offset = 7

# Draw a circle at each fixation point on the overlay and enumerate them
for i, (x, y) in enumerate(zip(x_coords_transformed, y_coords_transformed)):
    print(f'Drawing text for point {i+1} at coordinates ({int(x)}, {int(y)})')
    # Draw the outer black circle
    cv2.circle(viz_img, (int(x), int(y)), radius=5, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)
    
    # Add enumeration text
    #cv2.putText(viz_img, str(i+1), (int(x) + txt_offset, int(y)),
                #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

# Display the image with fixation points using Matplotlib
plt.imshow(viz_img)

# Show the name of the image in the title and the subject number, block number and trial number
plt.title(f'Image: {img_path.name}')
plt.axis('off')  # Hide the axis
plt.show()


##### 2.1.2.2 Visualization of Fixations - Case 2: If you want to create a fixation map for the first fixations(>50ms) of the first prestentation of the specific img for every participants 

In [ ]:
######## Eliminate the fixations <50ms from the df_filtered DataFrame (as suggested by Rothkegel et al (2019))########
# Eliminate fixations with durations < 50ms

# Load df_filtered from the previous cell
df_filtered = df_filtered
print(df_filtered)

# Debug check. Ensure the 'Fixation Duration' column is numeric. In the DataFrame, the column is stored as a float 
# indicating the duration of the fixation in s
df_filtered['Fixation Duration'] = pd.to_numeric(df_filtered['Fixation Duration'], errors='coerce')

# Eliminate fixations with durations < 50ms = < 0.05s (since the fixation duration is stored in seconds in the DataFrame)
df_filtered = df_filtered[df_filtered['Fixation Duration'] >= 0.05]

# Select the first fixation for each subject on the first presentation of the image
df_first_fixations = df_filtered.groupby(['Subject Number', 'Image Id']).first().reset_index()

# List the amount of rows of the filtered DataFrame - indicating the amount of fixations in the trial
print(len(df_first_fixations))
print(df_first_fixations)

# Save the first fixations DataFrame to a CSV file
df_first_fixations.to_csv(save_dir / f'firstFixations_nsd_stimulus_id_{img_num}.csv', index=False)

# If saved send a confirmation message
print(f'Saved first fixations to {save_dir / f"firstFixations_nsd_stimulus_id_{img_num}"}')

# Only load x and y coordinates from the filtered DataFrame
x_coords = df_first_fixations['Fixation X Coordinate'].values
y_coords = df_first_fixations['Fixation Y Coordinate'].values

# Print the x and y coordinates of the fixations with a duration of at least 50ms
print(x_coords)
print(y_coords)

# Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080)
# to the position on the image (e.g. 258 x 258)

# Get the original and cropped image dimensions
original_img_size = 425
cropped_img_size = 258

# Calculate the offset for centering the image on a 1920 x 1080 screen
offset_x = (1920 - cropped_img_size) / 2
offset_y = (1080 - cropped_img_size) / 2

# Adjust the fixation coordinates by subtracting the offset
x_coords = x_coords - offset_x
y_coords = y_coords - offset_y

# Scale the adjusted coordinates to match the cropped image dimensions
x_coords = (original_img_size / cropped_img_size) * x_coords
y_coords = (original_img_size / cropped_img_size) * y_coords

# Create a new DataFrame with the transformed coordinates so that the saliency map can be plotted with
# the correct fixation positions
df_first_fixations_transformed = df_first_fixations.copy()
df_first_fixations_transformed['Fixation X Coordinate Transformed'] = x_coords
df_first_fixations_transformed['Fixation Y Coordinate Transformed'] = y_coords

# Using OpenCV to draw circles on the image, and imitating transparency
viz_img = img.copy()
txt_offset = 7

# Draw a circle at each fixation point on the overlay and enumerate them
for i, (x, y) in enumerate(zip(x_coords, y_coords)):
    print(f'Drawing text for point {i+1} at coordinates ({int(x)}, {int(y)})')
    # Draw the outer black circle
    cv2.circle(viz_img, (int(x), int(y)), radius=5, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)
    
    # Add enumeration text
    #cv2.putText(viz_img, str(i+1), (int(x) + txt_offset, int(y)),
    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

# Display the image with fixation points using Matplotlib
plt.imshow(viz_img)

# Show the name of the image in the title and the subject number, block number and trial number
plt.title(f'Frist Fixations: {img_path.name}')
plt.axis('off')  # Hide the axis
plt.show()

# Save the visualization image with fixation points to a png file
save_path = save_dir / f'firstFixations_nsd_stimulus_id_{img_num}' 
cv2.imwrite(f'{save_path}.png', cv2.cvtColor(viz_img, cv2.COLOR_RGB2BGR))

# If saved send a confirmation message
print(f'Saved visualization to {save_path}.png')

############### Create the same visualization for the first fixation of each subject on the first presentation of the image with enumerations of the fixations ###############
# Draw a circle at each fixation point on the overlay and enumerate them
for i, (x, y) in enumerate(zip(x_coords, y_coords)):
    print(f'Drawing text for point {i+1} at coordinates ({int(x)}, {int(y)})')
    # Draw the outer black circle
    cv2.circle(viz_img, (int(x), int(y)), radius=5, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)
    
    # Add enumeration text
    cv2.putText(viz_img, str(i+1), (int(x) + txt_offset, int(y)),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

# Display the image with fixation points using Matplotlib
plt.imshow(viz_img)

# Show the name of the image in the title and the subject number, block number and trial number
plt.title(f'First fixations of: {img_path.name}')
plt.axis('off')  # Hide the axis
plt.show()

# Save the visualization image with fixation points to a png file
save_path = save_dir / f'firstFixations_nsd_stimulus_id_{img_num}' 
cv2.imwrite(f'{save_path}.png', cv2.cvtColor(viz_img, cv2.COLOR_RGB2BGR))

# If saved send a confirmation message
print(f'Saved visualization to {save_path}.png')



## 2.2. Estimation of AOI - Saliency Maps 

This code creates a saliency map (heat map), estimating the most salient areas a paticipant looked at. 

The saliency map is calculated via:
- Kernel density estimation (non-parametric way to infer the shape of a distribution from data, often visualized as smooth curves) 
- The random variable in the case of the NSD-Coco-saliency project is the saliency of a pixel (area) of the image 

Within this code to generate the saliency maos the code: 

- 1. Estimates the bandwidth(bw-ml or bw-ls) 
- 2. Creates a KDE object with the estimated bandwidth 
- 3. Calculates the probability density function (PDF) for each pixel of an image 
- 4. Generates the saliency maps by plotting the PDF values 
- 5. Overlay the saliency map on the actual image (By load the image and plotting the saliency map on top of it)



In [ ]:
x_coords.min()

### 2.2.1 Create a Saliency Map for All Fixations in an Image. 

In [ ]:
# Using the fixation map created in e.g firstFixations_nsd_stimulus_id_3857.csv, we can create a heatmap of the fixations on the image
# Do I have to use the first fixations or can I use all fixations for the generation of the saliency map?

# Use all fixations of the participants while perceive the image (first time?). Because this will provide a more comprehensive representatopn of the AOI's across all participants 
# and an accurate saliency map.

### Load the relevant data to generate a saliency map for the image (e.g. nsd_stimulus_id_3050.jpg) ###

# Load the image using cv2 - OpenCV
img = cv2.imread(str(img_path))

# Check if the image was loaded correctly
if img is None:
    raise ValueError("Image not found or unable to load.")

# Convert the image from BGR to RGB (OpenCV loads images in BGR format)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080), 
# was already done in the previous cells and is stored in df_transformed

# df_transformed contains the adjusted fixation points (c1, c2)
c1 = df_transformed['Fixation X Coordinate Transformed'].values
c2 = df_transformed['Fixation Y Coordinate Transformed'].values

# Estimate the bandwidth using KDEMultivariate with 'cv_ml' method
dens_u = sm.nonparametric.KDEMultivariate(data=[c1, c2], var_type='cc', bw='cv_ml')

# Create a meshgrid of the image dimensions
x_sz, y_sz = 425, 425
Ys, Xs = np.mgrid[0:x_sz, 0:y_sz]

# Prepare the data for prediction
data_predict = np.vstack([Xs.ravel(), Ys.ravel()]).T

# Calculate the log density values for each pixel
# log_density = kde.score_samples(data_predict)
density = dens_u.pdf(data_predict)

print(density.sum())

# Convert log density to density
#density = np.exp(log_density)

# Print the density values for each pixel
print(f"Density-values: {density}")

# Reshape the density values to match the image dimensions
saliency_map = density.reshape(x_sz, y_sz)

# Normalize the saliency map to have values between 0 and 1
# saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())

# Print the values of the saliency map
print(f"Saliency Map: {saliency_map}")

################### Save the saliency map to a CSV file ###################

# Save saliency map values to a CSV file
np.savetxt(save_dir / f'saliency_map_nsd_stimulus_id_{img_num}.csv', saliency_map, delimiter=',')
# If saved send a confirmation message
print(f'Saved saliency map to {save_dir / f"saliency_map_nsd_stimulus_id_{img_num}.csv"}')

# Print the location where the value is 1 in the saliency map
print(np.where(saliency_map == 1))

###########################################################################

# Display the saliency map
plt.imshow(saliency_map, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Saliency Map - All fixations: ' + img_id, fontsize= 10)
# Save the plot to a png file
save_path = save_dir / f'saliency_map_all_fixations_nsd_stimulus_id_{img_num}'
plt.savefig(f'{save_path}.png')

plt.show()

################### Overlay the saliency map on the image ###################

# Plot the image
plt.imshow(img, cmap='gray', interpolation='nearest')

# Overlay the saliency map with a trasparency of alpha = 0.7, TODO: Change to 0.5 if the code works s
plt.imshow(saliency_map, cmap='hot', alpha=0.6, interpolation='nearest')

# Add a colorbar and title
plt.colorbar()
plt.title('Saliency Map - All fixations: ' + img_id, fontsize=10)

# Save the plot to a png file
save_path = save_dir / f'saliency_map_overlay_all_fixations_nsd_stimulus_id_{img_num}'
plt.savefig(f'{save_path}.png')
# if saved send a confirmation message
print(f'Saved overlay to {save_path}.png')

# Shows the image in the console 
plt.show()

In [ ]:
saliency_map.shape

### 2.2.2 Now create the saliency maps just for the first fixations during the first presentation of an image!


In [ ]:
### Load the relevant data to generate a saliency map for an image of shared1000 (e.g. nsd_stimulus_id_3050.jpg) ###
#Used img ids are stored in the shared1000.tsv.txt file

# Load the image using cv2 - OpenCV
img = cv2.imread(str(img_path))

# Check if the image was loaded correctly
if img is None:
    raise ValueError("Image not found or unable to load.")

# Convert the image from BGR to RGB (OpenCV loads images in BGR format)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Linear transformation of the fixation x- and y-coordinates indicating the position on the screen (1920 x 1080), 
# was already done in the previous cells and is stored in df_transformed

# df_first?fixations?transformed contains the adjusted fixation points (c1, c2)
c1 = df_first_fixations_transformed['Fixation X Coordinate Transformed'].values
c2 = df_first_fixations_transformed['Fixation Y Coordinate Transformed'].values

# Print the x and y coordinates of the fixations
#print(f"Transformed fixation x value: {c1}")
#print(f"Transformed fixation z value: {c2}")

# Estimate the bandwidth using KDEMultivariate with 'cv_ml' method
dens_u = sm.nonparametric.KDEMultivariate(data=[c1, c2], var_type='cc', bw='cv_ml')

# Create a meshgrid of the image dimensions
x_sz, y_sz = 425, 425
Ys, Xs = np.mgrid[0:x_sz, 0:y_sz]

# Prepare the data for prediction
data_predict = np.vstack([Xs.ravel(), Ys.ravel()]).T

# Calculate the log density values for each pixel
# log_density = kde.score_samples(data_predict)
density = dens_u.pdf(data_predict)

print(density.sum())

# Convert log density to density
#density = np.exp(log_density)

# Print the density values for each pixel
print(f"Density-values: {density}")

# Reshape the density values to match the image dimensions
saliency_map = density.reshape(x_sz, y_sz)

# Normalize the saliency map to have values between 0 and 1 / Normalization is not desired for the saliency map, to be able to see intergroup differences
#saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())

# Print the values of the saliency map
print(f"Saliency Map: {saliency_map}")

###########################################################################

# Display the saliency map
plt.imshow(saliency_map, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('First Fixations - Saliency Map of: ' + img_id, fontsize=8)

# Save the plot to a png file
save_path = save_dir / f'saliency_map_first_fixations_nsd_stimulus_id_{img_num}'
plt.savefig(f'{save_path}.png')

plt.show()

################### Overlay the saliency map on the image ##################

# Plot the image
plt.imshow(img, cmap='gray', interpolation='nearest')

# Overlay the saliency map with a trasparency of alpha = 0.7, TODO: Change to 0.5 if the code works s
plt.imshow(saliency_map, cmap='hot', alpha=0.6, interpolation='nearest')

# Add a colorbar and title
plt.colorbar()
plt.title('First Fixations - Saliency Map of: ' + img_id, fontsize=8)

# Save the plot to a png file
save_path = save_dir / f'saliency_map_overlay_first_fixations_nsd_stimulus_id_{img_num}'
plt.savefig(f'{save_path}.png')

plt.show()